In [12]:
# Import Dependencies
import pandas as pd
import datetime as dt

In [13]:
# Read in Monthly Sales Price Data
price = pd.read_excel('Data/Med:Avg Sales Price.xls', sheet_name='Price ')
# Clean up DataFrame
price.columns = price.iloc[1]
price = price.iloc[2:]
# Remove timestamp from Period column
price['Period'] = pd.to_datetime(price['Period'], errors='coerce')
# Insert Median prices into empty Average values
price['Average'].loc[2:145] = price['Median '].loc[2:145]
# Drop Median column
price.drop(columns='Median ', inplace=True)
# Set Period as index
price.set_index('Period', inplace=True)
price = price.rename_axis('Period', axis='columns')
price.index.name = None
# Drop Null Values
price = price.dropna()

/var/folders/zq/0hmyg29903n1xkl833j8x3h00000gn/T/ipykernel_48039/1690424211.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Average'].loc[2:145] = price['Median '].loc[2:145]


In [14]:
# Read in Monthly Sales Quantity Data
quantity = pd.read_excel('Data/Sold by Region.xls', sheet_name='Reg Sold')

In [15]:
# Clean up df
quantity = quantity.iloc[7:]
quantity.rename(columns={'Unnamed: 1': 'Quantity'}, inplace= True)
quantity.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], inplace= True)

In [16]:
# Drop nulls
quantity = quantity.dropna()

In [17]:
quantity['Houses Sold by Region'] = pd.to_datetime(quantity['Houses Sold by Region'], errors='coerce')

In [18]:
quantity.rename(columns={'Houses Sold by Region': 'Period'}, inplace= True)

In [19]:
quantity.set_index('Period', inplace=True)

In [20]:
quantity['Quantity'] = quantity['Quantity'] * 1000

In [21]:
quantity

,Quantity
Period,
1963-01-01,42000
1963-02-01,35000
1963-03-01,44000
1963-04-01,52000
1963-05-01,58000
...,...
2021-12-01,61000
2022-01-01,70000
2022-02-01,72000
